In [2]:
from ALNSv2 import Solution
import pickle
from os import listdir, path
from os.path import join

# Test solutions

In [67]:
def unpickle_files(path):
    files = []
    for f in listdir(path):
        try:
            raw_file = open(join(path, f), 'rb')
            data = pickle.load(raw_file)
            files.append(data)
        except TypeError:
            logging.WARN(f"File {f} is not a pickleable object. SKIP")
    return files

solutions = unpickle_files("C:\\Users\\manuf\\OneDrive\\Dokumente\\Universitaet\\Masterthesis\\data\\vrpldtt_fontaine\\solution")
data = unpickle_files("C:\\Users\\manuf\\OneDrive\\Dokumente\\Universitaet\\Masterthesis\\data\\vrpldtt_fontaine\\data")

In [68]:
c203_id = 8
c203 = data[c203_id]
c203_sol = solutions[c203_id]

data_obj = c203
sol_obj = c203_sol

In [69]:
d_time = 0
prev_node_id = 0
for customer_id in sol_obj.solution[0]:
    load_level = int(sol_obj.loads[customer_id]/15)
    
    t_time = data_obj.time_cube[load_level][prev_node_id][customer_id+1]
    print(load_level, t_time)
    d_time += t_time
    
    prev_node_id = customer_id+1
    
d_time += data_obj.time_cube[0][prev_node_id][0]

9 5.265486725663812
6 0.856439704675979
1 0.5544


# Analyse: Where did we go wrong?

In [74]:
def get_load_lvl(load, interval):
    print(load)
    return int((load-1)/interval)


def get_route_tt(route, sol_obj, data_obj):
    tt = 0
    prev_nid = 0
    for cid in route:
        # get load level
        load_lvl = get_load_lvl(sol_obj.loads[cid], 15)
        # node to node
        tt += data_obj.time_cube[load_lvl][prev_nid][cid+1]
        prev_nid = cid+1

    tt += data_obj.time_cube[0][prev_nid][0]
    return tt

def get_solution_tt(sol_obj, data_obj):
    ttt = 0
    for route in sol_obj.solution:
        ttt += get_route_tt(route, sol_obj, data_obj)
        # Back to depot
    return ttt

get_solution_tt(sol_obj, data_obj)

147.5
92.5
22.5
125.0
57.5
105.0
35.0
127.5
65.0
45.0
130.0
117.5
100.0
40.0
127.5
110.0
65.0
147.5
97.5
35.0


45.00717081365909

In [71]:
[load%15 for load in sol_obj.loads]

[5.0,
 12.5,
 0.0,
 5.0,
 7.5,
 0.0,
 7.5,
 5.0,
 10.0,
 10.0,
 10.0,
 5.0,
 7.5,
 12.5,
 5.0,
 5.0,
 12.5,
 2.5,
 7.5,
 12.5]

In [19]:
sum(np.array(sol_obj.route_times)-np.array(route_service_times))

AttributeError: 'ALNSv2.Solution' object has no attribute 'route_times'

In [7]:
sol_obj.

[43.38383838383839,
 77.99404761904762,
 10.489247311827956,
 51.46604938271605,
 54.093567251461984]

In [8]:
[data_obj.start_window[route[0]] for route in sol_obj.solution]

[46.0, 79.0, 11.0, 54.0, 57.0]

In [9]:
# Why the fuck are the start times not always beginning - travel time?
# Also: Why would it every be? (Reconsider approach)

# REALIZATION: Order makes a differences (because of slope)

In [24]:
import pandas as pd
path = "C:\\Users\\manuf\\Documents\\Repositories\\metaheuristic-vrpldtt\\data\\vrpldtt_pirmin\\cases\\C303.csv"

data = pd.read_csv(path, index_col = 0)

In [25]:
data

,x,y,elevation,demand,tw a,tw b,s,0,1,2,...,11,12,13,14,15,16,17,18,19,20
0,40.44090,-79.98593,273.2,NaN,NaN,NaN,NaN,0.000,0.216,0.582,...,0.739,0.936,1.157,1.080,0.983,0.812,0.615,0.673,0.382,0.381
1,40.44263,-79.98713,279.9,67.5,11.0,90.0,5.0,0.216,0.000,0.367,...,0.808,1.004,1.084,0.865,1.059,0.921,0.803,0.890,0.597,0.597
2,40.44492,-79.98631,306.0,57.5,14.0,42.0,5.0,0.582,0.367,0.000,...,0.813,1.009,0.935,0.709,1.064,0.926,0.806,0.892,0.626,0.849
3,40.44590,-79.98400,317.8,45.0,32.0,80.0,5.0,0.800,0.585,0.230,...,0.665,0.862,0.705,0.479,0.908,0.750,0.662,0.852,0.804,1.030
4,40.44698,-79.98146,322.8,20.0,8.0,85.0,5.0,1.046,0.831,0.476,...,0.419,0.616,0.458,0.232,0.661,0.574,0.612,0.802,1.050,1.172
5,40.44548,-79.98164,319.4,62.5,54.0,111.0,5.0,0.809,0.665,0.509,...,0.377,0.573,0.480,0.254,0.629,0.471,0.383,0.573,0.806,0.943
6,40.44427,-79.98086,303.0,70.0,57.0,115.0,5.0,0.659,0.591,0.596,...,0.228,0.424,0.505,0.404,0.480,0.321,0.233,0.423,0.656,0.793
7,40.44374,-79.98254,304.8,17.5,45.0,91.0,5.0,0.600,0.425,0.430,...,0.383,0.579,0.659,0.560,0.634,0.496,0.355,0.545,0.596,0.822
8,40.44345,-79.98464,300.2,35.0,9.0,92.0,5.0,0.459,0.244,0.248,...,0.564,0.760,0.840,0.662,0.816,0.677,0.536,0.632,0.428,0.654
9,40.44231,-79.98327,286.4,40.0,20.0,110.0,5.0,0.292,0.480,0.460,...,0.469,0.666,0.887,0.789,0.713,0.543,0.346,0.432,0.288,0.514


In [30]:
data.loc[:,"0":].round(decimals=5).to_csv("C:\\Users\\manuf\\Documents\\Repositories\\metaheuristic-vrpldtt\\data\\tmp.csv", index=False)

In [11]:
elevations = list(data.loc[:, "elevation"])

In [12]:
# Get slope and distance matrix
def get_elevation_matrix(elevations):
    """
    Get the elevation matrix for a list elevations (simple)

    Annotation:
    We report the elevation in kms so that it is inline with the km reporting of the distances

    """
    nr_nodes = len(elevations)        
    elevation_matrix = [[0 for i in range(nr_nodes)] for j in range(nr_nodes)]

    for i in range(nr_nodes):
        for j in range(i+1, nr_nodes):
            # altitude difference to get from i to j
            elevation_distance = (elevations[j] - elevations[i])
            elevation_matrix[i][j] = elevation_distance
            elevation_matrix[j][i] = -elevation_distance # obviously the elevation is shifted now
    return elevation_matrix

In [14]:
data2 = pd.DataFrame(get_elevation_matrix(elevations))

In [22]:
data2.round(decimals=5).to_csv("C:\\Users\\manuf\\Documents\\Repositories\\metaheuristic-vrpldtt\\data\\tmp_elev.csv", index=False)

In [23]:
data2

,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,0.0,6.7,32.8,44.6,49.6,46.2,29.8,31.6,27.0,13.2,...,25.4,30.0,44.3,37.8,48.2,33.3,13.8,0.9,11.4,-8.9
1,-6.7,0.0,26.1,37.9,42.9,39.5,23.1,24.9,20.3,6.5,...,18.7,23.3,37.6,31.1,41.5,26.6,7.1,-5.8,4.7,-15.6
2,-32.8,-26.1,0.0,11.8,16.8,13.4,-3.0,-1.2,-5.8,-19.6,...,-7.4,-2.8,11.5,5.0,15.4,0.5,-19.0,-31.9,-21.4,-41.7
3,-44.6,-37.9,-11.8,0.0,5.0,1.6,-14.8,-13.0,-17.6,-31.4,...,-19.2,-14.6,-0.3,-6.8,3.6,-11.3,-30.8,-43.7,-33.2,-53.5
4,-49.6,-42.9,-16.8,-5.0,0.0,-3.4,-19.8,-18.0,-22.6,-36.4,...,-24.2,-19.6,-5.3,-11.8,-1.4,-16.3,-35.8,-48.7,-38.2,-58.5
5,-46.2,-39.5,-13.4,-1.6,3.4,0.0,-16.4,-14.6,-19.2,-33.0,...,-20.8,-16.2,-1.9,-8.4,2.0,-12.9,-32.4,-45.3,-34.8,-55.1
6,-29.8,-23.1,3.0,14.8,19.8,16.4,0.0,1.8,-2.8,-16.6,...,-4.4,0.2,14.5,8.0,18.4,3.5,-16.0,-28.9,-18.4,-38.7
7,-31.6,-24.9,1.2,13.0,18.0,14.6,-1.8,0.0,-4.6,-18.4,...,-6.2,-1.6,12.7,6.2,16.6,1.7,-17.8,-30.7,-20.2,-40.5
8,-27.0,-20.3,5.8,17.6,22.6,19.2,2.8,4.6,0.0,-13.8,...,-1.6,3.0,17.3,10.8,21.2,6.3,-13.2,-26.1,-15.6,-35.9
9,-13.2,-6.5,19.6,31.4,36.4,33.0,16.6,18.4,13.8,0.0,...,12.2,16.8,31.1,24.6,35.0,20.1,0.6,-12.3,-1.8,-22.1
